# Importing the dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%time
!unzip -q drive/MyDrive/dataset/dataset_zip.zip -d ./extracted_audios

CPU times: user 465 ms, sys: 63.1 ms, total: 528 ms
Wall time: 1min 23s


In [ ]:
# Importing the dependencies
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm

In [ ]:
# Importing the mappings csv file
mappings = 'drive/MyDrive/dataset/filename_to_classid_mapping.csv'
dataset = 'extracted_audios'

In [ ]:
labels = pd.read_csv(mappings, names = ['filenames', 'class_id'])

In [ ]:
labels

,filenames,class_id
0,0001.wav,1
1,0002.wav,1
2,0003.wav,1
3,0004.wav,1
4,0005.wav,1
...,...,...
1607,1608.wav,5
1608,1609.wav,5
1609,1610.wav,5
1610,1611.wav,5


# Feature Extraction

**Extracting MFCC features**

In [ ]:
def MFCC_features(file_name):
  audio, sample_rate = librosa.load(dataset + '/' + file_name)

  desired_length = 661500

  if len(audio) < desired_length:
    audio = np.pad(audio, (0, desired_length - len(audio)), mode = 'constant')

  MFCC = librosa.feature.mfcc(y = audio, sr = sample_rate, n_mfcc = 13)
  MFCC_scaled = np.mean(MFCC.T, axis = 0)

  return MFCC_scaled

In [ ]:
# Create a numpy matrix to store the mfcc vectors
X = np.zeros([1612, 13])
# A vector to store the output labels
Y = np.zeros(len(labels))

In [ ]:
# Extract the features
for index, rows in tqdm(labels.iterrows()):
  file_name = rows['filenames']
  class_id = rows['class_id']

  MFCC = MFCC_features(file_name)
  X[index] = MFCC
  Y[index] = class_id

1612it [02:55,  9.21it/s]


In [ ]:
X.shape

(1612, 13)

In [ ]:
# Transform the output labels from range (1 - 6) to (0 - 5)
y_transform = Y - 1

# CLASSIFICATION

In [ ]:
# Import the depencies
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [ ]:
# Perform the train test split
x_train, x_test, y_train, y_test = train_test_split(X, y_transform, test_size = 0.20)

In [ ]:
x_train.shape

(1289, 13)

In [ ]:
y_train.shape

(1289,)

In [ ]:
x_train[0]

array([-3.77247681e+02,  1.67994201e+00, -1.04700584e+01, -5.24070621e-01,
        7.72588062e+00,  1.17524347e+01, -3.31129223e-01, -8.42742324e-01,
        1.90315700e+00,  1.01613941e+01,  1.12753177e+00, -2.53947973e+00,
       -5.83370638e+00])

# Building Model


**Build model**


In [ ]:

#   create model
model = keras.Sequential()

#   2 LSTM layers
model.add(keras.layers.LSTM(128, input_shape = (13, 1), return_sequences = "True"))
model.add(keras.layers.LSTM(64))

#   Dense layer
model.add(keras.layers.Dense(64, activation = "tanh"))
model.add(keras.layers.Dropout(0.3))

#   output layer
model.add(keras.layers.Dense(6, activation = "softmax"))



In [ ]:
model.compile(
    optimizer = 'Adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 13, 128)           66560     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 390       
                                                                 
Total params: 120518 (470.77 KB)
Trainable params: 120518 (470.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 100)

Epoch 1/100
41/41 [==============================] - 7s 27ms/step - loss: 1.5365 - accuracy: 0.4057 - val_loss: 1.3296 - val_accuracy: 0.4985
Epoch 2/100
41/41 [==============================] - 0s 8ms/step - loss: 1.2940 - accuracy: 0.5260 - val_loss: 1.2077 - val_accuracy: 0.5449
Epoch 3/100
41/41 [==============================] - 0s 8ms/step - loss: 1.1928 - accuracy: 0.5593 - val_loss: 1.1284 - val_accuracy: 0.5480
Epoch 4/100
41/41 [==============================] - 0s 10ms/step - loss: 1.0812 - accuracy: 0.6059 - val_loss: 1.0119 - val_accuracy: 0.6409
Epoch 5/100
41/41 [==============================] - 0s 10ms/step - loss: 0.9723 - accuracy: 0.6509 - val_loss: 0.9328 - val_accuracy: 0.6780
Epoch 6/100
41/41 [==============================] - 0s 11ms/step - loss: 0.8967 - accuracy: 0.6843 - val_loss: 0.8053 - val_accuracy: 0.7245
Epoch 7/100
41/41 [==============================] - 0s 10ms/step - loss: 0.8210 - accuracy: 0.7145 - val_loss: 0.7891 - val_accuracy: 0.7028
Epoch 8/

**Saved Models**

In [ ]:
estimator = model.evaluate(x_test, y_test)

11/11 [==============================] - 0s 4ms/step - loss: 0.1673 - accuracy: 0.9659


In [ ]:
predicted = model.predict(x_train)
predicted

41/41 [==============================] - 1s 3ms/step


array([[5.5891252e-07, 5.7424927e-06, 9.2322829e-05, 1.3664733e-04,
        7.6855431e-06, 9.9975711e-01],
       [3.2086493e-05, 3.5986080e-04, 9.0676266e-01, 9.2210598e-02,
        1.8004634e-04, 4.5476804e-04],
       [1.7484494e-04, 1.3193162e-03, 4.3650296e-01, 5.6050175e-01,
        4.1705219e-04, 1.0840649e-03],
       ...,
       [2.4776803e-06, 2.8371608e-06, 9.9966264e-01, 3.2708817e-04,
        4.0284208e-06, 8.6442566e-07],
       [1.1697433e-07, 5.0703777e-05, 8.4438070e-04, 4.9733175e-05,
        3.5979967e-06, 9.9905151e-01],
       [2.2494038e-04, 9.9803180e-01, 1.5215379e-04, 1.2938931e-03,
        2.2200542e-05, 2.7499514e-04]], dtype=float32)

In [ ]:
predicted[0]

In [ ]:
label = np.argmax(predicted[0])

In [ ]:
label

In [ ]:
y_test[0]

In [ ]:
y_predicted = [np.argmax(i) for i in predicted]

In [ ]:
y_test[:5]

In [ ]:
y_predicted[:5]

In [ ]:
cm = tf.math.confusion_matrix(labels = y_test, predictions = y_predicted)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sn
plt.figure(figsize = (10, 7))
sn.heatmap(cm, annot = True, fmt = 'd')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_predicted))

In [ ]:
import scienceplots

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix
l = ['1', '2', '3', '4', '5', 'empty']


cm = confusion_matrix(y_test,y_predicted)

fig, axis = plt.subplots()
sn.heatmap(cm, annot=True, fmt='d', ax=axis);  #annot=True to annotate cells, ftm='g' to disable scientific notation

# labels, title and ticks
axis.autoscale(tight=True)
axis.set_xlabel('Predicted labels')
axis.set_ylabel('True labels')
axis.set_title('Confusion Matrix')
axis.xaxis.set_ticklabels(l)
axis.yaxis.set_ticklabels(l)

plt.show()

In [ ]:
with plt.style.context(['science', 'ieee', 'custom.mplstyle']):
    fig, axis = plt.subplots()
    sn.heatmap(cm, annot=True, fmt='d', ax=axis)
    axis.autoscale(tight=True)
    axis.set_xlabel('Predicted labels')
    axis.set_ylabel('True labels')
    axis.set_title('Confusion Matrix')
    axis.xaxis.set_ticklabels(l)
    axis.yaxis.set_ticklabels(l)
    fig.savefig('con.jpeg', dpi=600, pad_inches=0.5)
    plt.show()

In [ ]:

def plot_history(history):
  fig, axis = plt.subplots(2)

  #create accuracy subplot
  axis[0].plot(history.history["accuracy"], label = "train_accuracy")
  axis[0].plot(history.history["val_accuracy"], label = "test_accuracy")
  axis[0].set_ylabel("Accuracy")
  axis[0].legend(loc = "lower right")
  axis[0].set_title("accuracy eval")

  #create error subplot
  axis[1].plot(history.history["loss"], label = "train_error")
  axis[1].plot(history.history["val_loss"], label = "test_error")
  axis[1].set_xlabel("Epoch")
  axis[1].set_ylabel("Loss")
  axis[1].legend(loc = "upper right")
  axis[1].set_title("Error eval")


In [ ]:
plot_history(history)

In [ ]:
with plt.style.context(['science', 'ieee', 'accloss.mplstyle']):
    fig, axis = plt.subplots(2, 1)

    #create accuracy subplot
    axis[0].plot(history.history["accuracy"], label = "train_accuracy", color='mediumblue', linewidth=0.7)
    axis[0].plot(history.history["val_accuracy"], label = "test_accuracy", color='orange', linewidth=0.7, linestyle='solid')
    axis[0].set_ylabel("Accuracy")
    axis[0].set_xlabel("Epochs")
    axis[0].legend(loc = "lower right")
    axis[0].set_title("(a)")

    #create error subplot
    axis[1].plot(history.history["loss"], label = "train_error", color='mediumblue', linewidth=0.7)
    axis[1].plot(history.history["val_loss"], label = "test_error", color='orange', linewidth=0.7, linestyle='solid')
    axis[1].set_xlabel("Epochs")
    axis[1].set_ylabel("Loss")
    axis[1].legend(loc = "upper right")
    axis[1].set_title("(b)")

    plt.tight_layout()

    fig.savefig('accloss.png', dpi=600, pad_inches=0.3)

    plt.show()

In [ ]:
import numpy as np

# Assuming you have a confusion matrix in the form of a NumPy array
confusion_matrix = cm

# Calculate precision, recall, F1 score, and accuracy for each class
precision = np.diag(confusion_matrix) / np.sum(confusion_matrix, axis=0)
recall = np.diag(confusion_matrix) / np.sum(confusion_matrix, axis=1)
f1_score = 2 * (precision * recall) / (precision + recall)
accuracy = np.sum(np.diag(confusion_matrix)) / np.sum(confusion_matrix)

# Calculate overall precision, recall, F1 score, and accuracy
overall_precision = np.mean(precision)
overall_recall = np.mean(recall)
overall_f1_score = np.mean(f1_score)
overall_accuracy = accuracy

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)
print("Accuracy:", accuracy)

print("\nOverall Accuracy:", overall_accuracy)
print("Overall Precision:", overall_precision)
print("Overall Recall:", overall_recall)
print("Overall F1 Score:", overall_f1_score)

In [ ]:
model

In [ ]:
with plt.style.context(['science', 'ieee']):
    keras.utils.plot_model(model, show_shapes=True, show_layer_names=True, to_file='architecture.png', dpi=600, show_layer_activations=True) #to_file='test.png'
    # plt.savefig('test.png', dpi=300)
    plt.show()